In [1]:
# Evaluate the ML model

import tensorflow as tf
from tensorflow import keras
import os
import glob
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib as mpl
from utils import plot_loss, encoder_files_to_tensors, get_best_model_timestamp
from utils import load_model_data_new, unnormalize_params, assess_decoder
from utils import sample_files
from models import EncoderMulti, mse_loss_encoder, mse_loss_decoder

data_dir = '/eos/user/k/kiliakis/tomo_data/datasets'
dataset_percent = 0.5
normalization = 'minmax'
num_Turns_Case = 50+1

var_names = ['phEr', 'enEr', 'bl',
             'inten', 'Vrf', 'mu', 'VrfSPS']


2022-10-26 17:31:26.597049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 17:31:26.733677: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-26 17:31:26.737198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kiliakis/install/lib:/usr/lib/x86_64-linux-gnu
2022-10-26 17:31:26.737206: I tensorflow/st

In [2]:
# Initialize parameters
import time

# Initialize train/ test / validation paths
ML_dir = os.path.join(data_dir, 'ML_data')
TESTING_PATH = os.path.join(ML_dir, 'TESTING')
assert os.path.exists(TESTING_PATH)


# First the training data
file_names = sample_files(TESTING_PATH, dataset_percent, keep_every=num_Turns_Case)

start_t = time.time()
# read input, divide in features/ label, create tensors
x_test, y_test = encoder_files_to_tensors(file_names, normalization=normalization)
total_time = time.time() - start_t
print(f'Elapsed time: {total_time:.3f}, Per file: {total_time/len(file_names):.3f}')


2022-10-26 17:31:38.773344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kiliakis/install/lib:/usr/lib/x86_64-linux-gnu
2022-10-26 17:31:38.773362: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-26 17:31:38.773406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sy153): /proc/driver/nvidia/version does not exist
2022-10-26 17:31:38.773620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Elapsed time: 42.648, Per file: 0.062


In [11]:
# Model to load
# timestamp = get_best_model_timestamp('./trials', model='enc')
# timestamp = '2022_10_07_15-18-55'
# timestamp = '2022_09_30_17-46-45'
# timestamp = '2022_10_24_12-28-57'
timestamp = '2022_10_26_15-29-45'

print(timestamp)

# Initialize directories
trial_dir = os.path.join('./trials/', timestamp)
weights_dir = os.path.join(trial_dir, 'weights')
plots_dir = os.path.join(trial_dir, 'plots')
assert os.path.exists(weights_dir)
os.makedirs(plots_dir, exist_ok=True)

models = {}
# load the model
for file in os.listdir(weights_dir):
    var_name = file.split('encoder_')[1].split('.h5')[0]
    models[var_name] = keras.models.load_model(os.path.join(weights_dir, file))


2022_10_26_15-29-45


In [12]:
test_losses = {}
for var_name, model in models.items():
    idx = var_names.index(var_name)
    print(f'------ Evaluating {var_name} ------')
    test_loss = model.evaluate(x_test, tf.gather(y_test, idx, axis=1))
    test_losses[var_name] = test_loss

for name, mse in test_losses.items():
    print(f'{name}:\t{mse:.4f}')

print(f'\nMean loss: {np.mean([v for v in test_losses.values()]):.4f}')


------ Evaluating phEr ------
22/22 [==============================] - 0s 4ms/step - loss: 1.3309e-04
------ Evaluating enEr ------
22/22 [==============================] - 0s 4ms/step - loss: 3.6850e-04
------ Evaluating bl ------
22/22 [==============================] - 0s 4ms/step - loss: 8.8329e-04
------ Evaluating inten ------
22/22 [==============================] - 0s 4ms/step - loss: 0.0851
------ Evaluating Vrf ------
22/22 [==============================] - 0s 4ms/step - loss: 2.6305e-04
------ Evaluating mu ------
22/22 [==============================] - 0s 4ms/step - loss: 0.0047
------ Evaluating VrfSPS ------
22/22 [==============================] - 0s 4ms/step - loss: 0.0029
phEr:	0.0001
enEr:	0.0004
bl:	0.0009
inten:	0.0851
Vrf:	0.0003
mu:	0.0047
VrfSPS:	0.0029

Mean loss: 0.0135


In [ ]:
test_pred = np.zeros(shape=(len(x_test), len(models)), dtype=float)
for var_name, model in models.items():
    idx = var_names.index(var_name)
    print(f'------ Generating predictions for {var_name} ------')
    test_pred[:, idx] = model.predict(x_test, verbose=False).reshape(-1)

mses = mean_squared_error(y_test, test_pred, multioutput='raw_values')

for name, mse in zip(var_names, mses):
    print(f'{name}:\t{mse:.4f}')

print(f'\nMean loss: {np.mean(mses):.4f}')


In [ ]:
# Also generate MSE histogram
# print(valid_pred.shape)
square_errors = ((y_test - test_pred) * (y_test - test_pred))

fig, axes = plt.subplots(ncols=1, nrows=len(var_names), sharex=False, 
sharey=True, figsize=(8, 16))
for i, name in enumerate(var_names):
    hist,edges = np.histogram(square_errors[:, i], bins=10, density=False)
    hist = hist / len(square_errors[:, i])
    print(name, hist)
    ax = axes[i]
    plt.sca(ax)
    plt.bar(np.arange(len(hist)), hist, width=0.8)
    plt.title(name)
    edges = [f'{e:.4f}' for e in edges]
    plt.xticks(np.arange(len(hist)), edges[1:])
    plt.xlabel('Error')
    plt.ylabel('Density')
    plt.tight_layout()


In [ ]:

y_test_unnorm = unnormalize_params(
    y_test[:, 0], y_test[:, 1], y_test[:, 2],
    y_test[:, 3], y_test[:, 4], y_test[:, 5],
    y_test[:, 6], normalization=normalization)

y_pred_unnorm = unnormalize_params(
    test_pred[:, 0], test_pred[:, 1], test_pred[:, 2],
    test_pred[:, 3], test_pred[:, 4], test_pred[:, 5],
    test_pred[:, 6], normalization=normalization)

diffs = np.array(y_pred_unnorm).T - np.array(y_test_unnorm).T
print(diffs.shape)
fig, axes = plt.subplots(ncols=1, nrows=len(var_names), sharex=False,
                         sharey=True, figsize=(8, 16))
fig.suptitle(f'Model id: {timestamp}')

plt.sca(axes[0])
plt.hist(diffs[:, 0], bins=100, range=(-50, 50))
plt.xlabel('Phase Error Diff [deg]')
plt.sca(axes[1])
plt.hist(diffs[:, 1], bins=100, range=(-100, 100))
plt.xlabel('Energy Error Diff [MeV]')
plt.sca(axes[2])
plt.hist(diffs[:, 2]*1e12, bins=100, range=(-500, 500))
plt.xlabel('Bunch length Diff [ps]')
plt.sca(axes[3])
plt.hist(diffs[:, 3]*1e-10, bins=100, range=(-10.5, 10.5))
plt.xlabel('Intensity diff [1e9 prot]')
plt.sca(axes[4])
plt.hist(diffs[:, 4], bins=100, range=(-0.3, 0.3))
plt.xlabel('V_rf diff [MV]')
plt.sca(axes[5])
plt.hist(diffs[:, 5], bins=100, range=(-0.5, 0.5))
plt.xlabel('mu diff [a.u.]')
plt.sca(axes[6])
plt.hist(diffs[:, 6], bins=100, range=(-2, 2))
plt.xlabel('V_rf SPS diff [MV]')
plt.tight_layout()

